In [ ]:
import cv2
import numpy as np
import math
import sys # Để thoát chương trình nếu có lỗi

# --- Tham số có thể điều chỉnh ---
IMAGE_PATH = "/Users/namtran/Downloads/att.jNzfE_t_vU1Hv3jirwwYFH1m5aO6zquCTxFo7G7BEoU.JPG"  # <<< THAY ĐỔI ĐƯỜNG DẪN NÀY
THRESHOLD_VALUE = 173    # Giá trị ngưỡng (thử thay đổi hoặc dùng THRESH_OTSU)
USE_OTSU = False                  # Đặt là True để dùng Otsu thay vì giá trị ngưỡng cố định
THRESHOLD_TYPE = cv2.THRESH_BINARY # Hoặc cv2.THRESH_BINARY_INV nếu vật tối trên nền sáng
BLUR_KERNEL_SIZE = (3, 3)         # Kích thước bộ lọc làm mờ (nên là số lẻ)
MIN_CONTOUR_AREA = 20      # Bỏ qua các contour quá nhỏ (chống nhiễu)
# ---------------------------------

def calculate_orientation(moments):
    """Tính góc định hướng từ moments trung tâm."""
    if moments['m00'] == 0:
        return 0 # Tránh chia cho 0

    # Moment trung tâm cần thiết để tính góc
    mu20 = moments['mu20']
    mu02 = moments['mu02']
    mu11 = moments['mu11']

    # Công thức tính góc (radian)
    angle_rad = 0.5 * math.atan2(2 * mu11, mu20 - mu02)
    return angle_rad

def main():
    # 1. Đọc ảnh
    image = cv2.imread(IMAGE_PATH)
    if image is None:
        print(f"LỖI: Không thể đọc file ảnh tại: {IMAGE_PATH}")
        sys.exit(1) # Thoát chương trình

    # 2. Chuyển sang ảnh xám
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Làm mờ nhẹ để giảm nhiễu
    gray_blurred = cv2.GaussianBlur(gray, BLUR_KERNEL_SIZE, 0)

    # 3. Phân ngưỡng
    thresh = None
    if USE_OTSU:
        # Dùng phương pháp Otsu để tự động tìm ngưỡng tối ưu
        ret, thresh = cv2.threshold(gray_blurred, 0, 255, THRESHOLD_TYPE | cv2.THRESH_OTSU)
        print(f"Ngưỡng Otsu được sử dụng: {ret}")
    else:
        # Dùng giá trị ngưỡng cố định
        ret, thresh = cv2.threshold(gray_blurred, THRESHOLD_VALUE, 255, THRESHOLD_TYPE)
        print(f"Ngưỡng cố định được sử dụng: {THRESHOLD_VALUE}")

    # Hiển thị ảnh nhị phân để kiểm tra
    cv2.imshow("Thresholded Image", thresh)

    # 4. Tìm đường bao
    # cv2.findContours trả về contours và hierarchy
    # Đảm bảo tương thích với các phiên bản OpenCV khác nhau
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours: # Kiểm tra xem list contours có rỗng không
        print("LỖI: Không tìm thấy đường bao nào sau khi phân ngưỡng.")
        print("Hãy thử điều chỉnh giá trị ngưỡng hoặc kiểm tra ảnh đầu vào.")
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        sys.exit(1)

    # 5. Chọn đường bao lớn nhất (hoặc phù hợp nhất)
    valid_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > MIN_CONTOUR_AREA]

    if not valid_contours:
        print(f"LỖI: Không tìm thấy đường bao nào đủ lớn (diện tích > {MIN_CONTOUR_AREA}).")
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        sys.exit(1)

    # Chọn contour có diện tích lớn nhất trong các contour hợp lệ
    object_contour = max(valid_contours, key=cv2.contourArea)
    max_area = cv2.contourArea(object_contour)
    print(f"Số đường bao hợp lệ tìm thấy: {len(valid_contours)}")
    print(f"Đường bao lớn nhất có diện tích: {max_area:.2f}")

    # 6. Tính toán Moments
    M = cv2.moments(object_contour)

    # 7. Tính tọa độ trọng tâm
    cx, cy = 0, 0
    if M['m00'] != 0:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        centroid = (cx, cy)
        print(f"Tọa độ trọng tâm (pixel): ({cx}, {cy})")
    else:
        print("CẢNH BÁO: Diện tích đường bao bằng 0, không thể tính trọng tâm.")
        centroid = None # Đặt là None nếu không tính được

    # 8. Tính góc (Orientation)
    angle_rad = calculate_orientation(M)
    angle_deg = math.degrees(angle_rad) # Chuyển sang độ
    print(f"Góc định hướng (độ): {angle_deg:.2f}")

    # 9. Hiển thị kết quả trên ảnh gốc
    result_image = image.copy() # Tạo bản sao để vẽ lên

    # Vẽ đường bao của vật thể được chọn (màu xanh lá)
    cv2.drawContours(result_image, [object_contour], -1, (0, 255, 0), 2)

    # Vẽ trọng tâm (vòng tròn màu đỏ) nếu tính được
    if centroid:
        cv2.circle(result_image, centroid, 5, (0, 0, 255), -1)

        # Vẽ đường thẳng chỉ hướng (màu vàng)
        line_length = 70 # Độ dài đường chỉ hướng
        # Tính điểm cuối của đường chỉ hướng
        p2_x = int(cx + line_length * math.cos(angle_rad))
        p2_y = int(cy + line_length * math.sin(angle_rad))
        cv2.line(result_image, centroid, (p2_x, p2_y), (0, 255, 255), 2)

    # Hiển thị tọa độ và góc lên ảnh
    coord_text = f"Centroid: ({cx}, {cy})" if centroid else "Centroid: N/A"
    angle_text = f"Angle: {angle_deg:.2f} deg"
    text_org_coord = (10, result_image.shape[0] - 40) # Vị trí góc dưới trái
    text_org_angle = (10, result_image.shape[0] - 20)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.6
    color = (255, 255, 0) # Màu vàng nhạt
    thickness = 1
    cv2.putText(result_image, coord_text, text_org_coord, font, font_scale, color, thickness, cv2.LINE_AA)
    cv2.putText(result_image, angle_text, text_org_angle, font, font_scale, color, thickness, cv2.LINE_AA)

    # Hiển thị ảnh kết quả
    cv2.imshow("Ket qua Bai 2 - Python", result_image)
    print("\nNhấn phím bất kỳ trên cửa sổ ảnh để thoát...")
    cv2.waitKey(0) # Chờ người dùng nhấn phím bất kỳ
    cv2.destroyAllWindows() # Đóng tất cả cửa sổ OpenCV

if __name__ == "__main__":
    main()

Ngưỡng cố định được sử dụng: 173
Số đường bao hợp lệ tìm thấy: 5
Đường bao lớn nhất có diện tích: 37158.00
Tọa độ trọng tâm (pixel): (613, 403)
Góc định hướng (độ): 25.65

Nhấn phím bất kỳ trên cửa sổ ảnh để thoát...


In [ ]:
import cv2
import numpy as np
import math
import sys

# --- Tham số có thể điều chỉnh ---
IMAGE_PATH = "/Users/namtran/Downloads/att.jNzfE_t_vU1Hv3jirwwYFH1m5aO6zquCTxFo7G7BEoU.JPG"  # <<< THAY ĐỔI ĐƯỜNG DẪN NÀY
THRESHOLD_VALUE = 185
USE_OTSU = False
THRESHOLD_TYPE = cv2.THRESH_BINARY # Hoặc cv2.THRESH_BINARY_INV
BLUR_KERNEL_SIZE = (7, 7)
MIN_CONTOUR_AREA = 25

# --- Tham số hình thái học ---
USE_MORPHOLOGY = True             # Đặt là True để bật xử lý hình thái
MORPH_OPERATION = cv2.MORPH_CLOSE # Thử cv2.MORPH_CLOSE hoặc cv2.MORPH_OPEN
MORPH_KERNEL_SHAPE = cv2.MORPH_RECT # Hoặc cv2.MORPH_ELLIPSE
MORPH_KERNEL_SIZE = (5, 5)        # Kích thước kernel (nên là số lẻ)
# ---------------------------------

def calculate_orientation(moments):
    """Tính góc định hướng từ moments trung tâm."""
    if moments['m00'] == 0:
        return 0

    mu20 = moments['mu20']
    mu02 = moments['mu02']
    mu11 = moments['mu11']

    angle_rad = 0.5 * math.atan2(2 * mu11, mu20 - mu02)
    return angle_rad

def main():
    # 1. Đọc ảnh
    image = cv2.imread(IMAGE_PATH)
    if image is None:
        print(f"LỖI: Không thể đọc file ảnh tại: {IMAGE_PATH}")
        sys.exit(1)

    # 2. Chuyển sang ảnh xám
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Làm mờ nhẹ
    gray_blurred = cv2.GaussianBlur(gray, BLUR_KERNEL_SIZE, 0)

    # 3. Phân ngưỡng
    thresh = None
    if USE_OTSU:
        ret, thresh = cv2.threshold(gray_blurred, 0, 255, THRESHOLD_TYPE | cv2.THRESH_OTSU)
        print(f"Ngưỡng Otsu được sử dụng: {ret}")
    else:
        ret, thresh = cv2.threshold(gray_blurred, THRESHOLD_VALUE, 255, THRESHOLD_TYPE)
        print(f"Ngưỡng cố định được sử dụng: {THRESHOLD_VALUE}")

    cv2.imshow("1 - Thresholded Image", thresh) # Hiển thị ảnh gốc sau threshold

    # --- Áp dụng phép toán hình thái học (nếu bật) ---
    processed_image = thresh # Mặc định dùng ảnh threshold gốc
    if USE_MORPHOLOGY:
        # Tạo kernel (cấu trúc phần tử)
        kernel = cv2.getStructuringElement(MORPH_KERNEL_SHAPE, MORPH_KERNEL_SIZE)

        # Thực hiện phép toán hình thái
        morph_result = cv2.morphologyEx(thresh, MORPH_OPERATION, kernel)

        # (Tùy chọn) Có thể thực hiện thêm phép toán khác, ví dụ Opening sau Closing
        # kernel_open = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        # morph_result = cv2.morphologyEx(morph_result, cv2.MORPH_OPEN, kernel_open)

        print(f"Da ap dung phep toan hinh thai: {MORPH_OPERATION} voi kernel {MORPH_KERNEL_SIZE}")
        cv2.imshow("2 - Morphological Result", morph_result) # Hiển thị ảnh sau hình thái
        processed_image = morph_result # Sử dụng ảnh đã xử lý cho bước tiếp theo
    # ----------------------------------------------------

    # 4. Tìm đường bao (trên ảnh đã xử lý hình thái hoặc ảnh threshold gốc)
    contours, hierarchy = cv2.findContours(processed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        print("LỖI: Không tìm thấy đường bao nào.")
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        sys.exit(1)

    # 5. Chọn đường bao lớn nhất (hoặc phù hợp nhất)
    valid_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > MIN_CONTOUR_AREA]

    if not valid_contours:
        print(f"LỖI: Không tìm thấy đường bao nào đủ lớn (diện tích > {MIN_CONTOUR_AREA}).")
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        sys.exit(1)

    object_contour = max(valid_contours, key=cv2.contourArea)
    max_area = cv2.contourArea(object_contour)
    print(f"Số đường bao hợp lệ tìm thấy: {len(valid_contours)}")
    print(f"Đường bao lớn nhất có diện tích: {max_area:.2f}")

    # 6. Tính toán Moments
    M = cv2.moments(object_contour)

    # 7. Tính tọa độ trọng tâm
    cx, cy = 0, 0
    centroid = None
    if M['m00'] != 0:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        centroid = (cx, cy)
        print(f"Tọa độ trọng tâm (pixel): ({cx}, {cy})")
    else:
        print("CẢNH BÁO: Diện tích đường bao bằng 0.")

    # 8. Tính góc (Orientation)
    angle_rad = calculate_orientation(M)
    angle_deg = math.degrees(angle_rad)
    print(f"Góc định hướng (độ): {angle_deg:.2f}")

    # 9. Hiển thị kết quả trên ảnh gốc
    result_image = image.copy()
    cv2.drawContours(result_image, [object_contour], -1, (0, 255, 0), 2)

    if centroid:
        cv2.circle(result_image, centroid, 5, (0, 0, 255), -1)
        line_length = 70
        p2_x = int(cx + line_length * math.cos(angle_rad))
        p2_y = int(cy + line_length * math.sin(angle_rad))
        cv2.line(result_image, centroid, (p2_x, p2_y), (0, 255, 255), 2)

    coord_text = f"Centroid: ({cx}, {cy})" if centroid else "Centroid: N/A"
    angle_text = f"Angle: {angle_deg:.2f} deg"
    text_org_coord = (10, result_image.shape[0] - 40)
    text_org_angle = (10, result_image.shape[0] - 20)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.6
    color = (255, 255, 0)
    thickness = 1
    cv2.putText(result_image, coord_text, text_org_coord, font, font_scale, color, thickness, cv2.LINE_AA)
    cv2.putText(result_image, angle_text, text_org_angle, font, font_scale, color, thickness, cv2.LINE_AA)

    cv2.imshow("3 - Ket qua cuoi cung", result_image)
    print("\nNhấn phím bất kỳ trên cửa sổ ảnh để thoát...")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Ngưỡng cố định được sử dụng: 185
Da ap dung phep toan hinh thai: 3 voi kernel (5, 5)
Số đường bao hợp lệ tìm thấy: 3
Đường bao lớn nhất có diện tích: 36060.00
Tọa độ trọng tâm (pixel): (614, 403)
Góc định hướng (độ): 25.74

Nhấn phím bất kỳ trên cửa sổ ảnh để thoát...


In [ ]:
import cv2
import numpy as np
import math
import sys
import time # Thêm để tính FPS (tùy chọn)

# --- Tham số Camera ---
CAMERA_INDEX = 0                  # Thường là 0 cho camera mặc định, thử 1, 2 nếu có nhiều camera

# --- Tham số xử lý ảnh (Giữ nguyên hoặc điều chỉnh từ trước) ---
THRESHOLD_VALUE = 187
USE_OTSU = False
THRESHOLD_TYPE = cv2.THRESH_BINARY # Hoặc cv2.THRESH_BINARY_INV
BLUR_KERNEL_SIZE = (7, 7)
MIN_CONTOUR_AREA = 25

# --- Tham số hình thái học ---
USE_MORPHOLOGY = True             # Bật/tắt xử lý hình thái
MORPH_OPERATION = cv2.MORPH_CLOSE # Thử cv2.MORPH_CLOSE hoặc cv2.MORPH_OPEN
MORPH_KERNEL_SHAPE = cv2.MORPH_RECT # Hoặc cv2.MORPH_ELLIPSE
MORPH_KERNEL_SIZE = (5, 5)        # Kích thước kernel (nên là số lẻ)
# ---------------------------------

def calculate_orientation(moments):
    """Tính góc định hướng từ moments trung tâm."""
    if moments['m00'] == 0:
        return 0

    mu20 = moments['mu20']
    mu02 = moments['mu02']
    mu11 = moments['mu11']

    angle_rad = 0.5 * math.atan2(2 * mu11, mu20 - mu02)
    return angle_rad

def main():
    # 1. Mở camera
    cap = cv2.VideoCapture(CAMERA_INDEX)
    if not cap.isOpened():
        print(f"LỖI: Không thể mở camera index {CAMERA_INDEX}")
        sys.exit(1)

    print(f"Đã mở camera index {CAMERA_INDEX}. Nhấn 'ESC' để thoát.")

    # Lấy kích thước khung hình (nếu cần)
    # width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    # height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # print(f"Kích thước khung hình: {width}x{height}")

    # Tạo cửa sổ
    cv2.namedWindow("1 - Live Camera Feed")
    cv2.namedWindow("2 - Thresholded")
    if USE_MORPHOLOGY:
        cv2.namedWindow("3 - Morphological Result")
    cv2.namedWindow("4 - Final Result")

    prev_time = 0 # Để tính FPS

    # Tạo kernel hình thái một lần bên ngoài vòng lặp nếu dùng
    kernel = None
    if USE_MORPHOLOGY:
        kernel = cv2.getStructuringElement(MORPH_KERNEL_SHAPE, MORPH_KERNEL_SIZE)

    # 2. Vòng lặp xử lý từng khung hình
    while True:
        # Đọc khung hình từ camera
        ret, frame = cap.read()
        if not ret:
            print("LỖI: Không thể đọc khung hình từ camera. Kết thúc.")
            break

        # --- Tính FPS (tùy chọn) ---
        current_time = time.time()
        fps = 1 / (current_time - prev_time) if (current_time - prev_time) > 0 else 0
        prev_time = current_time
        # --- ---

        # Hiển thị khung hình gốc
        cv2.imshow("1 - Live Camera Feed", frame)

        # --- Bắt đầu xử lý ảnh ---
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray_blurred = cv2.GaussianBlur(gray, BLUR_KERNEL_SIZE, 0)

        thresh = None
        if USE_OTSU:
            ret_thresh, thresh = cv2.threshold(gray_blurred, 0, 255, THRESHOLD_TYPE | cv2.THRESH_OTSU)
        else:
            ret_thresh, thresh = cv2.threshold(gray_blurred, THRESHOLD_VALUE, 255, THRESHOLD_TYPE)

        cv2.imshow("2 - Thresholded", thresh)

        processed_image = thresh
        if USE_MORPHOLOGY and kernel is not None:
            morph_result = cv2.morphologyEx(thresh, MORPH_OPERATION, kernel)
            cv2.imshow("3 - Morphological Result", morph_result)
            processed_image = morph_result

        # Tìm đường bao
        # Tạo bản sao vì findContours có thể thay đổi ảnh đầu vào trong một số phiên bản OpenCV
        contours_input = processed_image.copy()
        contours, hierarchy = cv2.findContours(contours_input, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # --- Xử lý và vẽ kết quả nếu tìm thấy contour hợp lệ ---
        result_image = frame.copy() # Vẽ lên bản sao của khung hình gốc
        centroid = None
        angle_deg = 0.0
        object_found = False # Cờ để biết có tìm thấy vật thể không

        valid_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > MIN_CONTOUR_AREA]

        if valid_contours: # Chỉ xử lý nếu có contour hợp lệ
            object_contour = max(valid_contours, key=cv2.contourArea)
            M = cv2.moments(object_contour)

            if M['m00'] != 0:
                object_found = True # Tìm thấy và tính được moments
                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])
                centroid = (cx, cy)

                angle_rad = calculate_orientation(M)
                angle_deg = math.degrees(angle_rad)

                # Vẽ đường bao
                cv2.drawContours(result_image, [object_contour], -1, (0, 255, 0), 2)
                # Vẽ trọng tâm
                cv2.circle(result_image, centroid, 5, (0, 0, 255), -1)
                # Vẽ đường chỉ hướng
                line_length = 70
                p2_x = int(cx + line_length * math.cos(angle_rad))
                p2_y = int(cy + line_length * math.sin(angle_rad))
                cv2.line(result_image, centroid, (p2_x, p2_y), (0, 255, 255), 2)
        # --- Kết thúc xử lý contour ---

        # Hiển thị thông tin lên ảnh kết quả
        coord_text = f"Centroid: ({cx}, {cy})" if object_found else "Centroid: N/A"
        angle_text = f"Angle: {angle_deg:.2f} deg" if object_found else "Angle: N/A"
        fps_text = f"FPS: {fps:.1f}"
        cv2.putText(result_image, coord_text, (10, result_image.shape[0] - 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(result_image, angle_text, (10, result_image.shape[0] - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(result_image, fps_text, (10, result_image.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1, cv2.LINE_AA)

        cv2.imshow("4 - Final Result", result_image)

        # 3. Chờ phím nhấn để thoát
        key = cv2.waitKey(1) & 0xFF # Chờ 1ms, lấy 8 bit cuối
        if key == 27:  # 27 là mã ASCII của phím ESC
            print("Đã nhấn ESC, thoát chương trình.")
            break

    # 4. Giải phóng tài nguyên khi kết thúc
    cap.release()
    cv2.destroyAllWindows()
    print("Đã giải phóng camera và đóng cửa sổ.")

if __name__ == "__main__":
    main()

[ WARN:0@4.719] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Đã mở camera index 0. Nhấn 'ESC' để thoát.


In [ ]:
import cv2
import numpy as np
import math
import sys
import time

# --- Tham số Camera ---
CAMERA_INDEX = 0

# --- Tham số Xử lý Ảnh (Giữ nguyên hoặc điều chỉnh) ---
THRESHOLD_VALUE = 187
USE_OTSU = False
THRESHOLD_TYPE = cv2.THRESH_BINARY
BLUR_KERNEL_SIZE = (7, 7)
MIN_CONTOUR_AREA = 25

# --- Tham số Hình thái học ---
USE_MORPHOLOGY = True
MORPH_OPERATION = cv2.MORPH_CLOSE
MORPH_KERNEL_SHAPE = cv2.MORPH_RECT
MORPH_KERNEL_SIZE = (5, 5)

# --- Phím điều khiển ---
CAPTURE_KEY = ord('c') # Nhấn phím 'c' để chụp
# Hoặc CAPTURE_KEY = 32 # Dùng phím Spacebar

# ---------------------------------

def calculate_orientation(moments):
    """Tính góc định hướng từ moments trung tâm."""
    if moments['m00'] == 0: return 0
    mu20, mu02, mu11 = moments['mu20'], moments['mu02'], moments['mu11']
    return 0.5 * math.atan2(2 * mu11, mu20 - mu02)

def process_captured_image(captured_image):
    """Thực hiện toàn bộ quá trình xử lý trên ảnh đã chụp."""
    if captured_image is None:
        return None, None, None # Trả về None nếu không có ảnh

    print("--- Processing Captured Image ---")
    start_time = time.time()

    # --- Các bước xử lý giống như trước ---
    gray = cv2.cvtColor(captured_image, cv2.COLOR_BGR2GRAY)
    gray_blurred = cv2.GaussianBlur(gray, BLUR_KERNEL_SIZE, 0)

    thresh = None
    if USE_OTSU:
        ret_thresh, thresh = cv2.threshold(gray_blurred, 0, 255, THRESHOLD_TYPE | cv2.THRESH_OTSU)
        print(f"Otsu Threshold: {ret_thresh}")
    else:
        ret_thresh, thresh = cv2.threshold(gray_blurred, THRESHOLD_VALUE, 255, THRESHOLD_TYPE)
        print(f"Fixed Threshold: {THRESHOLD_VALUE}")

    thresh_display = cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR) # Để hiển thị
    cv2.putText(thresh_display, "Thresholded", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    processed_image = thresh
    morph_display = None
    if USE_MORPHOLOGY:
        kernel = cv2.getStructuringElement(MORPH_KERNEL_SHAPE, MORPH_KERNEL_SIZE)
        morph_result = cv2.morphologyEx(thresh, MORPH_OPERATION, kernel)
        morph_display = cv2.cvtColor(morph_result, cv2.COLOR_GRAY2BGR) # Để hiển thị
        cv2.putText(morph_display, "Morphology", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        processed_image = morph_result
        print(f"Applied Morphology: {MORPH_OPERATION}, Kernel: {MORPH_KERNEL_SIZE}")

    contours_input = processed_image.copy()
    contours, hierarchy = cv2.findContours(contours_input, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    result_image = captured_image.copy() # Vẽ lên bản sao của ảnh đã chụp
    centroid = None
    angle_deg = 0.0
    object_found = False

    valid_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > MIN_CONTOUR_AREA]
    print(f"Found {len(contours)} total contours, {len(valid_contours)} valid contours.")

    if valid_contours:
        object_contour = max(valid_contours, key=cv2.contourArea)
        M = cv2.moments(object_contour)
        if M['m00'] != 0:
            object_found = True
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            centroid = (cx, cy)
            angle_rad = calculate_orientation(M)
            angle_deg = math.degrees(angle_rad)
            print(f"Object Found - Centroid: ({cx}, {cy}), Angle: {angle_deg:.2f} deg")

            # Vẽ
            cv2.drawContours(result_image, [object_contour], -1, (0, 255, 0), 2)
            cv2.circle(result_image, centroid, 5, (0, 0, 255), -1)
            line_length = 70
            p2_x = int(cx + line_length * math.cos(angle_rad))
            p2_y = int(cy + line_length * math.sin(angle_rad))
            cv2.line(result_image, centroid, (p2_x, p2_y), (0, 255, 255), 2)
        else:
             print("Contour found, but area is zero.")
    else:
        print("No valid object contour found.")

    # Thêm text vào ảnh kết quả
    coord_text = f"Centroid: ({cx}, {cy})" if object_found else "Centroid: N/A"
    angle_text = f"Angle: {angle_deg:.2f} deg" if object_found else "Angle: N/A"
    cv2.putText(result_image, coord_text, (10, result_image.shape[0] - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1, cv2.LINE_AA)
    cv2.putText(result_image, angle_text, (10, result_image.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1, cv2.LINE_AA)

    end_time = time.time()
    print(f"Processing took: {end_time - start_time:.4f} seconds")
    print("--- Processing Finished ---")

    # Trả về các ảnh để hiển thị
    return thresh_display, morph_display, result_image
# ----------------------------------------------------------------------

def main():
    cap = cv2.VideoCapture(CAMERA_INDEX)
    if not cap.isOpened():
        print(f"LỖI: Không thể mở camera index {CAMERA_INDEX}")
        sys.exit(1)

    print(f"Camera mở. Nhấn '{chr(CAPTURE_KEY)}' để chụp ảnh, 'ESC' để thoát.")

    # Tạo cửa sổ
    cv2.namedWindow("1 - Live Camera Feed")
    cv2.namedWindow("2 - Analysis Results") # Một cửa sổ để hiển thị kết quả

    captured_image = None # Lưu ảnh đã chụp
    thresh_to_show = None # Lưu ảnh threshold của ảnh đã chụp
    morph_to_show = None  # Lưu ảnh morphology của ảnh đã chụp
    result_to_show = None # Lưu ảnh kết quả cuối cùng của ảnh đã chụp

    while True:
        ret, frame = cap.read()
        if not ret:
            print("LỖI: Không thể đọc khung hình.")
            break

        # Hiển thị live feed với hướng dẫn
        display_frame = frame.copy()
        cv2.putText(display_frame, f"Press '{chr(CAPTURE_KEY).upper()}' to Capture", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow("1 - Live Camera Feed", display_frame)

        # --- Xử lý phím nhấn ---
        key = cv2.waitKey(1) & 0xFF

        if key == 27: # ESC
            break
        elif key == CAPTURE_KEY:
            captured_image = frame.copy() # Lưu lại khung hình hiện tại
            print(f"\nẢnh đã được chụp! Bắt đầu xử lý...")
            # Gọi hàm xử lý ảnh đã chụp
            thresh_to_show, morph_to_show, result_to_show = process_captured_image(captured_image)

        # --- Hiển thị kết quả của ảnh ĐÃ CHỤP (nếu có) ---
        # Chúng ta sẽ ghép các ảnh kết quả lại để hiển thị trong một cửa sổ
        if result_to_show is not None:
            display_list = []
            # Lấy kích thước ảnh kết quả để tạo ảnh đen nếu cần
            h, w = result_to_show.shape[:2]

            # Thêm ảnh Threshold (nếu có)
            if thresh_to_show is not None:
                display_list.append(thresh_to_show)
            else: # Tạo ảnh đen nếu không có
                 display_list.append(np.zeros((h, w//2 if w > 100 else w , 3), dtype=np.uint8)) # Chiều rộng nhỏ hơn

            # Thêm ảnh Morphology (nếu có)
            if morph_to_show is not None:
                 display_list.append(morph_to_show)
            elif USE_MORPHOLOGY: # Chỉ tạo ảnh đen nếu hình thái học được bật
                 display_list.append(np.zeros((h, w//2 if w > 100 else w, 3), dtype=np.uint8))

            # Thêm ảnh kết quả cuối cùng
            display_list.append(result_to_show)

            # Ghép các ảnh lại theo chiều ngang
            analysis_display = cv2.hconcat(display_list)
            cv2.imshow("2 - Analysis Results", analysis_display)


    # Dọn dẹp
    cap.release()
    cv2.destroyAllWindows()
    print("Đã giải phóng camera và đóng cửa sổ.")

if __name__ == "__main__":
    main()


[ WARN:0@0.140] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Camera mở. Nhấn 'c' để chụp ảnh, 'ESC' để thoát.


qt.qpa.keymapper: Mismatch between Cocoa 'c' and Carbon '\x0' for virtual key 8 with QFlags<Qt::KeyboardModifier>(NoModifier)
qt.qpa.keymapper: Mismatch between Cocoa 'C' and Carbon '\x0' for virtual key 8 with QFlags<Qt::KeyboardModifier>(ShiftModifier)
qt.qpa.keymapper: Mismatch between Cocoa 'c' and Carbon '\x0' for virtual key 8 with QFlags<Qt::KeyboardModifier>(ControlModifier)
qt.qpa.keymapper: Mismatch between Cocoa 'c' and Carbon '\x0' for virtual key 8 with QFlags<Qt::KeyboardModifier>(ShiftModifier|ControlModifier)
qt.qpa.keymapper: Mismatch between Cocoa '\u00e7' and Carbon '\x0' for virtual key 8 with QFlags<Qt::KeyboardModifier>(AltModifier)
qt.qpa.keymapper: Mismatch between Cocoa '\u00c7' and Carbon '\x0' for virtual key 8 with QFlags<Qt::KeyboardModifier>(ShiftModifier|AltModifier)
qt.qpa.keymapper: Mismatch between Cocoa '\u00e7' and Carbon '\x0' for virtual key 8 with QFlags<Qt::KeyboardModifier>(ControlModifier|AltModifier)
qt.qpa.keymapper: Mismatch between Cocoa '\


Ảnh đã được chụp! Bắt đầu xử lý...
--- Processing Captured Image ---
Fixed Threshold: 187
Applied Morphology: 3, Kernel: (5, 5)
Found 81 total contours, 14 valid contours.
Object Found - Centroid: (644, 318), Angle: -76.06 deg
Processing took: 0.0090 seconds
--- Processing Finished ---
